In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import (
    Binarizer,
    StandardScaler,
    LabelBinarizer,
    OneHotEncoder,
)
from sklearn.linear_model import LogisticRegression
from sklearnex import patch_sklearn

patch_sklearn()
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
# from xgboost import XGBRFClassifier, XGBClassifier
from sklearn.neural_network import MLPClassifier
# import dtale
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

# sns.set()
pd.options.display.max_columns = 50
pd.options.plotting.backend = "matplotlib"
from sklearn import set_config

set_config(display="diagram")
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder,
    OrdinalEncoder,
    MinMaxScaler,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_graphviz
import warnings
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

warnings.filterwarnings("ignore")
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import (
    RandomForestClassifier,
    ExtraTreesClassifier,
    AdaBoostClassifier,
    HistGradientBoostingClassifier,
    GradientBoostingClassifier,
)
from sklearn.model_selection import (
    train_test_split,
    cross_validate,
    # StratifiedGroupKFold,

    StratifiedKFold,
)
import sigopt

from sklearn.naive_bayes import BernoulliNB, CategoricalNB, MultinomialNB, ComplementNB, GaussianNB
from sklearn.model_selection import cross_validate
nb_est = [CategoricalNB(), MultinomialNB(), ComplementNB(), GaussianNB()]
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.tree import export_graphviz, plot_tree
# import seaborn as sns
# import matplotlib.pyplot as plt
from tqdm import tqdm
# from xgboost import XGBClassifier
from joblib import parallel_backend
# from autosklearn.automl import AutoMLClassifier
import sklearn.metrics
dtype_info = {
    "v_1": "Binary",
    "v_26": "Binary",
    "v_11": "Binary",
    "v_14": "Binary",
    "v_30": "Binary",
    "v_28": "Binary",
    "v_9": "Binary",
    "v_27": "Binary",
    "v_32": "Nominal",
    "v_4": "Nominal",
    "v_3": "Nominal",
    "v_20": "Nominal",
    "v_21": "Nominal",
    "v_18": "Nominal",
    "v_25": "Nominal",
    "v_12": "Nominal",
    "v_31": "Ordinal",
    "v_15": "Ordinal",
    "v_19": "Ordinal",
    "v_13": "Ordinal",
    "v_33": "Ordinal",
    "v_17": "Ordinal",
    "v_29": "Ordinal",
    "v_23": "Ordinal",
    "v_6": "Ordinal",
    "v_24": "Ordinal",
    "v_10": "Ordinal",
    "v_5": "Ordinal",
    "v_22": "Ordinal",
    "v_0": "Ordinal",
    "v_16": "Ratio",
    "v_2": "Ratio",
    "v_8": "Ratio",
    "v_7": "Ratio",
    "v_39": "Ratio",
    "v_37": "Ratio",
    "v_38": "Ratio",
    "v_34": "Ratio",
    "v_40": "Ratio",
    "v_36": "Ratio",
    "v_35": "Ratio",
}
data = pd.read_csv('../data/train.csv',index_col=0,)
ordinal = [i for i in dtype_info if dtype_info[i] == "Ordinal"]
nominal = [i for i in dtype_info if dtype_info[i] == "Nominal"]
binary = [i for i in dtype_info if dtype_info[i] == "Binary"]
ratio = [i for i in dtype_info if dtype_info[i] == "Ratio"]
def categorise_data(data):
    ordinal_data = data.loc[:, ordinal]
    nominal_data = data.loc[:, nominal]
    binary_data = data.loc[:, binary]
    ratio_data = data.loc[:, ratio]
    return ordinal_data, nominal_data, binary_data, ratio_data
ordinal_data, nominal_data, binary_data, ratio_data = categorise_data(data)
def gen_train_test(data, p):
    Y = data.label
    X_2= Y_2 = Y[Y==2].index
    X_0= Y_0 = Y[Y==0].index
    X_1= Y_1 = Y[Y==1].index
    train_size = int(p*Y_2.shape[0])
    test_size = int((1-p)*Y_2.shape[0])
    
    train_idx_2 = np.random.choice(Y_2,(train_size,))
    train_idx_1 = np.random.choice(Y_1,(train_size,))
    train_idx_0 = np.random.choice(Y_0,(train_size,))
    train_idx = np.r_[train_idx_0, train_idx_1,train_idx_2]
    # train_idx.shape

    test_idx_2 = np.random.choice(np.setdiff1d(Y_2, train_idx_2),(test_size,))
    test_idx_1 = np.random.choice(np.setdiff1d(Y_1, train_idx_1),(test_size,))
    test_idx_0 = np.random.choice(np.setdiff1d(Y_0, train_idx_0),(test_size,))
    test_idx = np.r_[test_idx_0, test_idx_1,test_idx_2]
    # test_idx.shape
    return train_idx, test_idx
data__ = pd.read_parquet('../data/data_with_ridit.hdfs', engine='fastparquet')
import os
os.environ["SIGOPT_PROJECT"] = 'notebooks'

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [1]:
%reload_ext sigopt
%sigopt config

SigOpt API token (find at https://app.sigopt.com/tokens/info):

  ················································


Log Collection
	This will capture and upload the standard output and standard error of your
	Runs from the CLI and notebook cells so that you can view them on the SigOpt dashboard.
Enable log collection [y/N]:

  y


Notebook Cell Tracking
	This will record and upload the content of your notebook cells so that you can view them
	on the SigOpt dashboard.
Enable cell tracking [y/N]:

  y


In [5]:
# X_train, y_train = nom_dum.loc[train_idx,:], data['label'].loc[train_idx]
# X_train, y_train = nominal_data.loc[train_idx,:], data['label'].loc[train_idx]
# X_train, y_train = nom_dum.iloc[train_idx,imp_nom], data['label'].loc[train_idx]
# cv_results = cross_validate(dtc, X_train, y_train, scoring='f1_macro', return_train_score=True, n_jobs=-1, cv=10, return_estimator=True, )
# cv_results
train_idx, test_idx = gen_train_test(data, 0.75)
X_train, y_train = nominal_data.loc[train_idx,:].astype('category'), data['label'].loc[train_idx]
X_test, y_test = nominal_data.loc[test_idx,:].astype('category'), data["label"].loc[test_idx]

{'fit_time': array([0.14069819, 0.13676572, 0.14937949, 0.14447856, 0.09823632,
        0.15595841, 0.135391  , 0.15466595, 0.12151432, 0.12056446]),
 'score_time': array([0.00430202, 0.0066185 , 0.00564098, 0.02986598, 0.00554204,
        0.00778198, 0.00798416, 0.00675464, 0.0118072 , 0.00685859]),
 'estimator': [RandomForestClassifier(random_state=42),
  RandomForestClassifier(random_state=42),
  RandomForestClassifier(random_state=42),
  RandomForestClassifier(random_state=42),
  RandomForestClassifier(random_state=42),
  RandomForestClassifier(random_state=42),
  RandomForestClassifier(random_state=42),
  RandomForestClassifier(random_state=42),
  RandomForestClassifier(random_state=42),
  RandomForestClassifier(random_state=42)],
 'test_score': array([0.65279291, 0.66159978, 0.5947116 , 0.72323131, 0.6745203 ,
        0.69382614, 0.6381798 , 0.62606354, 0.58700523, 0.6462607 ]),
 'train_score': array([0.98494384, 0.98209191, 0.98280332, 0.98280873, 0.98424467,
        0.98352941,